In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
import keras.layers as L
from keras.datasets import fashion_mnist
from keras.applications import DenseNet121
from keras.applications import VGG16

In [2]:
def show(img):
    plt.imshow(img)

In [3]:
def create_exemplar_set(size, images, trained_model):
    #'images' has images all of class=y
    exemplars=[]
    for i in images:
        exemplars.append(calc_feature_vector(i, trained_model, layer_index))
    chosen_exemplars=exemplars[:size]
    mean_exemplar=sum(exemplars)/len(images)
    return chosen_exemplars, mean_exemplar

In [4]:
def create_custom_loss():
    gan_vs_nogan=keras.losses.BinaryCrossentropy(label_smoothing=0.1)
    type_of_gan=keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
    combined_loss=tf.add(gan_vs_nogan, type_of_gan)
    return combined_loss

 For multi-task learning, branching NN out
- one branch predicts the class of FashionMNIST object (classification)
- other branch predicts whether the object is a "top" or not (0/1 output)

In [147]:
def create_model(n_classes, input_dim):
    '''
        Creating categorical classification model
    '''
    
    inputs=L.Input((input_dim,input_dim,1))
    
    x=L.Conv2D(64, kernel_size=3, padding='same', strides=1)(inputs)
    x=L.LeakyReLU(0.2)(x)
    x=L.Conv2D(64, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    shared=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(shared)
    x=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    x=L.Flatten()(x)
    x=L.Dense(n_classes)(x)
    outputs=L.Softmax()(x)
    
    model_cla=keras.Model(inputs=inputs, outputs=outputs)
    model_cla.compile(optimizer=keras.optimizers.Adam(0.02), 
                  loss=keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])
    
    '''
        Creating binary classification model (a top/not a top)
    '''
    
    ip= L.Input(shape=shared.shape.as_list()[1:])
    x=L.LeakyReLU(0.2)(ip)
    x=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    x=L.Flatten()(x)
    op=L.Dense(1, activation=keras.activations.sigmoid)(x)

    model_bin=keras.Model(inputs=ip, outputs=op)
    model_bin.compile(optimizer=keras.optimizers.Adam(0.02), 
                      loss=keras.losses.BinaryCrossentropy(), 
                      metrics=['accuracy'])
    return model_cla, model_bin

In [148]:
modelc, modelb=create_model(10, 28)
modelc.summary()
modelb.summary()

Model: "model_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_45 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_387 (Conv2D)          (None, 28, 28, 64)        640       
_________________________________________________________________
leaky_re_lu_453 (LeakyReLU)  (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_388 (Conv2D)          (None, 28, 28, 64)        36928     
_________________________________________________________________
leaky_re_lu_454 (LeakyReLU)  (None, 28, 28, 64)        0         
_________________________________________________________________
max_pooling2d_224 (MaxPoolin (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_389 (Conv2D)          (None, 14, 14, 32)        184

In [149]:
x_train.shape

(60000, 28, 28, 1)

In [156]:
modelc.fit(x_train, y_train, epochs=5)

Epoch 1/5


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [85]:
def train_model(model, n_epochs, x_train, y_train):
    model.fit(x=x_train, 
              y=y_train, 
              epochs=n_epochs)
    return model

In [12]:
def calc_feature_vector(image, model, layer_index):
    #extract feature vector from a layer of ALREADY-TRAINED model
    extractor=keras.Model(model.inputs(), [model.layers[layer_index].output])
    vector=extractor(image)
    #divide vector by norm
    return vector

In [16]:
def classify(X, mean_exemplar_sets):    
    image_vector=calc_feature_vector(X)
    #exemplar set closest to X feature map is chosen
    return None

In [153]:
(x_train, y_train), (x_test, y_test)=tf.keras.datasets.fashion_mnist.load_data()

In [154]:
x_train=x_train.reshape((x_train.shape[0], 28,28,1))
x_test=x_test.reshape((x_test.shape[0],28,28,1))
y_train=keras.utils.to_categorical(y_train)
y_test=keras.utils.to_categorical(y_test)

In [155]:
x_train.shape

(60000, 28, 28, 1)

In [113]:
x_train

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [38]:
labeldict={}
labeldict[0]=1
labeldict[1]=0
labeldict[2]=1
labeldict[3]=1
labeldict[4]=1
labeldict[5]=0
labeldict[6]=1
labeldict[7]=0
labeldict[8]=0
labeldict[9]=0

In [133]:
y_train_multi=np.ndarray((0,2))
for i in range(len(y_train)):
    y_train_multi=np.append(y_train_multi,[[y_train[i],labeldict[y_train[i]]]], axis=0)

In [134]:
y_train_multi

array([[9., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [3., 1.],
       [0., 1.],
       [5., 0.]])